In [47]:
import pandas as pd

In [48]:
# http://www.statistikdatabasen.scb.se/pxweb/sv/ssd/START__HE__HE0202/HE0202T06/table/tableViewLayout1/
# ?rxid=ddccda22-c8fc-461a-9d50-c27bf861c804
living_cost = pd.read_csv(r'./living_cost_sweden/consumption.csv')
living_cost['2015'] = (living_cost['2015']*1000/12).astype(int)
living_cost.columns = ['Location', 'Cost of living (2015)']
living_cost

,Location,Cost of living (2015)
0,00 Riket,20783
1,0010 Stor-Stockholm,23791
2,0020 Stor-Göteborg,20650
3,0042 Kommuner med > 75000 inv (exkl Stor-Stock...,19258
4,0043 Kommuner med < 75000 inv (exkl Stor-Stock...,20516


In [49]:
# http://www.lonestatistik.se/loner.asp/yrke/Programmerare-1001
wages = pd.read_csv(r'./living_cost_sweden/it_wages.csv')
wages['Medellön'] = wages['Medellön'].str.replace(' kr', '').apply(pd.to_numeric)
wages = wages[['Län', 'Medellön']]
wages.columns = ['County', 'Mean wage']

In [50]:
# https://en.wikipedia.org/wiki/List_of_municipalities_of_Sweden
# http://wikitable2csv.ggor.de/
# Loading and preprocessing
muni = pd.read_csv(r'./living_cost_sweden/municipalities.csv')
muni['County'] = muni['County'].str.replace(' County', '')
muni['Municipality'] = muni['Municipality'].str.replace(' Municipality', '')
muni = muni[['Municipality', 'County', 'Population']]

In [51]:
muni.sample(10)

,Municipality,County,Population
122,Lilla Edet,Västra Götaland,12766
226,Torsby,Värmland,12037
278,Ängelholm,Skåne,39821
235,Töreboda,Västra Götaland,8972
37,Essunga,Västra Götaland,5529
118,Lerum,Västra Götaland,39203
172,Partille,Västra Götaland,36097
251,Vellinge,Skåne,33806
27,Danderyd,Stockholm,32074
94,Karlskoga,Örebro,29766


In [52]:
# Filling consumption values for each Municipality
muni.loc[:, 'Cost of living'] = 20516
muni.loc[muni['Population'] > 75000, 'Consumption'] = 19258
muni.loc[muni['County'] == 'Stockholm', 'Consumption'] = 23791
muni.loc[muni['Municipality'] == 'Göteborg', 'Consumption'] = 20650

# Filling wages values for each County
muni['Wages'] = 0
for lon in wages['County']:
    muni.loc[muni['County'] == lon, 'Wages'] = wages.loc[wages['County'] == lon, 'Mean wage'].values

# Smart cash is the wage normalized by the consumption index or cost of living
muni['smart_cash'] = muni['Wages']/muni['Cost of living']

# Mean by County and rescaling
smart_cash = muni.groupby('County')['smart_cash'].mean().reset_index()
spread = (smart_cash['smart_cash'].max() - smart_cash['smart_cash'].min())
smart_cash['smart_cash_idx'] = (smart_cash['smart_cash'] - smart_cash['smart_cash'].min()) / spread

In [53]:
smart_cash

,County,smart_cash,smart_cash_idx
0,Blekinge,0.001420,0.373616
1,Dalarna,0.001414,0.359131
2,Gotland,0.001259,0.030626
3,Gävleborg,0.001244,0.000000
4,Halland,0.001336,0.195447
5,Jämtland,0.001604,0.763994
6,Jönköping,0.001608,0.771133
7,Kalmar,0.001515,0.575168
8,Kronoberg,0.001455,0.446353
9,Norrbotten,0.001392,0.313606
